# TESTING

Import Statements

In [ ]:
import os
import re
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm import tqdm

Device Configuration

In [ ]:
# Suppress tokenizer warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Detect device
device = torch.device("cpu")
print(f"Using device: {device}")
torch.set_num_threads(16)

Using device: cpu


## Loading and preprocessing according to Model Architecture

In [ ]:
test_df = pd.read_csv('test.tsv', sep='\t')  # replace with your file name

test_df['title'] = test_df['title'].astype(str).str.strip()
test_df['subject'] = test_df['subject'].astype(str).str.strip()
test_df['text'] = test_df['text'].astype(str).str.strip()

test_df['content'] = test_df['title'] + ' ' + test_df['subject'] + ' ' + test_df['text']

def clean_for_bert(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.encode('ascii', 'ignore').decode()
    text = text.lower()
    return text.strip()

test_df['content'] = test_df['content'].apply(clean_for_bert)

test_texts = list(test_df['content'])
test_labels = list(test_df['label'])

## Saving Tokenized text for fast retrieval

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("saved_model")

encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)

torch.save({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': torch.tensor(test_labels)
}, "test_tokenized.pt")
print("Saved pre-tokenized test data.")

Saved pre-tokenized test data.


In [ ]:
data = torch.load("test_tokenized.pt")

dataset = TensorDataset(data['input_ids'], data['attention_mask'], data['labels'])
test_loader = DataLoader(dataset, batch_size=32)

## Testing Loop

In [67]:
model = AutoModelForSequenceClassification.from_pretrained("saved_model").to(device)

model.eval()
test_loss = 0
preds_test = []
true_labels_test = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        logits = outputs.logits
        loss = outputs.loss

        test_loss += loss.item() * labels.size(0)

        predictions = torch.argmax(logits, dim=-1)
        preds_test.extend(predictions.cpu().numpy())
        true_labels_test.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader.dataset)
test_acc = accuracy_score(true_labels_test, preds_test)

print(f"Test Loss: {avg_test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Testing: 100%|██████████| 259/259 [43:42<00:00, 10.13s/it]

Test Loss: 0.0000 | Test Accuracy: 1.0000


## Classification Report

In [68]:
from sklearn.metrics import classification_report

print(classification_report(true_labels_test, preds_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4284
           1       1.00      1.00      1.00      3983

    accuracy                           1.00      8267
   macro avg       1.00      1.00      1.00      8267
weighted avg       1.00      1.00      1.00      8267

